In [1]:
%load_ext autoreload
%autoreload 2

import os
import torch
import time
import json
import numpy as np

from src.sdf_convertion import reconstruct_voxels
from src.simulation import simulate

In [2]:
# Load the model

import sys
sys.path.insert(1, "external/MeshSDF")
from lib.models.decoder import DeepSDF

DEVICE="cuda:2"

experiment_dir = "runs/wolfish_e256/"

specs = json.load(open(os.path.join(experiment_dir, "specs.json")))
train_mapping = json.load(open(specs["TrainSplit"]))
data_mapping = json.load(open("/".join(specs["TrainSplit"].split("/")[:-1] + ["mapping.json"])))

# Load the model
decoder = torch.nn.DataParallel(DeepSDF(specs["CodeLength"],  **specs["NetworkSpecs"]), device_ids=[DEVICE])
saved_model_state = torch.load(
    os.path.join(experiment_dir, "ModelParameters", "latest.pth"), map_location=DEVICE
)
decoder.load_state_dict(saved_model_state["model_state_dict"])
decoder = decoder.eval()

# Load latent codes
orig_latents = torch.load(os.path.join(experiment_dir, "LatentCodes/latest.pth"), map_location=DEVICE)["latent_codes"]["weight"]

In [3]:
from pathlib import Path
import sys
import time
import math
import random
import copy
from collections import deque
from tqdm import trange, tqdm

import scipy
import scipy.optimize
import numpy as np

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import trimesh
import pyvista as pv
import imageio

from diffpd.fem import DeformableHex, HydrodynamicsStateForceHex
from diffpd.sim import Sim
from diffpd.nn import OpenFoldController
from diffpd import transforms
from diffpd.mesh import MeshHex

In [4]:
seed = 42
pv.start_xvfb()

# Mesh parameters.
length = 20
dx = 1. / length

# Hydrodynamics parameters.
rho = 1e1
v_water = [0, 0, 0]   # Velocity of the water.
# Cd_points = (angle, coeff) pairs where angle is normalized to [0, 1].
Cd_points = np.array([
    [0.0, 0.05],
    [0.4, 0.05],
    [0.7, 1.85],
    [1.0, 2.05],
]) # * 1.0
# Ct_points = (angle, coeff) pairs where angle is normalized to [-1, 1].
Ct_points = np.array([
    [-1, -0.8],
    [-0.3, -0.5],
    [0.3, 0.1],
    [1, 2.5],
])

# FEM parameters.
youngs_modulus = 1e6
poissons_ratio = 0.45
average_density = 1e1
dt = 3.33e-3
method = 'pd_eigen'
options = {
    'max_pd_iter': 2000, 'max_ls_iter': 10, 'abs_tol': 1e-4, 'rel_tol': 1e-3,
    'verbose': 0, 'thread_ct': 64, 'use_bfgs': 1, 'bfgs_history_size': 10
}


# def simulate(voxels, num_frames=100):
#     random.seed(seed)
#     np.random.seed(seed)
#     torch.manual_seed(seed)
#     # torch.set_default_dtype(torch.float64)

# Optimisation

In [5]:
latent = orig_latents[14]
optimizer = torch.optim.Adam([latent], lr=1e-2)
voxels = reconstruct_voxels(decoder, latent, N=[64, 32, 32]).numpy()

/mnt/arteml/classes/68420/src/sdf_convertion.py:125: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(sdf_values < 0., dtype=torch.float32)


In [6]:
def simmm(voxels):
    ## Inint simulator
    num_frames = 20

    shape = voxels.shape
    rest_mesh = MeshHex.load(voxels.clone().detach().numpy(), dx=dx)
    transform = []
    transform.append(transforms.AddStateForce(
        'hydrodynamics', [rho] + v_water + Cd_points.ravel().tolist() + Ct_points.ravel().tolist() + rest_mesh.boundary.ravel().tolist()))

    actuator_scale = 0.04
    actuator_height = int(shape[2] * actuator_scale)
    actuator_width =  int(shape[1] * actuator_scale)
    print(10)

    all_muscles = []
    shared_muscles = []
    for z in range(int(shape[2] / 2) - actuator_height, int(shape[2] / 2) + actuator_height):
        muscle_pair = []
        for y in range(int(shape[1] / 2) - actuator_width, int(shape[1] / 2) + actuator_width):
            indices = rest_mesh.cell_indices[int(0.45 * shape[0]):int(0.5 * shape[0]), y, z].tolist()
            transform.append(transforms.AddActuationEnergy(1e6, [1.0, 0.0, 0.0], indices))
            muscle_pair.append(indices)

            # print(indices)
        shared_muscles.append(muscle_pair)
    all_muscles.append(shared_muscles)

    transform = transforms.Compose(transform)

    deformable = DeformableHex(
        rest_mesh, density=average_density, dt=dt, method=method, options=options,)
    deformable = transform(deformable)

    dofs = deformable.dofs()
    act_dofs = deformable.act_dofs()

    q0 = torch.as_tensor(rest_mesh.vertices).view(-1).clone().detach().to(torch.float64)
    v0 = torch.zeros(dofs).detach().to(torch.float64)

    head_indices = rest_mesh.node_indices[0, 0, 0]

    sim = Sim(deformable)
    sim.add_default_pd_energies(['corotated'], youngs_modulus, poissons_ratio)

    voxel_mesh = q0.clone()
    voxel_mesh.requires_grad = True
    voxel_mesh = voxel_mesh.to(torch.float64)

    controller = OpenFoldController(
        deformable, all_muscles,
        num_steps=num_frames,
        segment_len=1,
        init_period=16.0,
        init_magnitude=128.0).to(torch.float64)

    a = None
    q, v = q0, v0

    qs, vs = [], []
    for a in controller():
        q, v = sim(q, v, a, shape=voxel_mesh)

        qs.append(q)
        vs.append(v)

    speed = torch.mean(torch.concatenate(vs).reshape(-1, 3)[:, 0])
    return speed

In [7]:
start_time = time.time()
voxels = reconstruct_voxels(decoder, latent, N=[64, 32, 32])

for it in range(1):
    # optimizer.zero_grad()
    
    # Forward
    speed = simmm(voxels)
    loss = -speed
    
    loss.backward()
    # optimizer.step()
    
    time_elapsed = time.time() - start_time
    print(f"#{it:4d} : Fish Speed: {speed:.4f}       Time Elapsed: {time_elapsed:.2f}", end='\n')

10
#   0 : Fish Speed: 0.0791       Time Elapsed: 7.53


In [10]:
voxels.grad